### Use this notebook for developing mlframe prototype 2

In [1]:
from commands import commands
commands.lscom()

Scripts location: /Users/peterbryzgalov/work/ML/mlframework/mlframe/commands/../scripts
test_nvidia_ubuntu 		Get versions of installed Nvidia drivers and CUDA
test_docker_ubuntu 		Get versions of Docker and nvidia-docker
install_docker_ubuntu 		Install Docker on Ubuntu
install_nvdocker_ubuntu 		Install nvidia-docker on Ubuntu
test 		Test script with sleep


In [2]:
commands.remote("DLbox","test")

Command file is /Users/peterbryzgalov/work/ML/mlframework/mlframe/commands/../scripts/test.sh


In [4]:
import multiprocessing
import subprocess
import os

In [120]:
# Class for executing commands in background processes.
# command_and_args must be a string with the command and arguments separated with spaces.
# It can be used for executing remote processes in background. For that purpose exec_remote.sh should be used.
class BashExecutor(multiprocessing.Process):
    debug = False
    def __init__(self, command_and_args, debug=False):
        super(BashExecutor,self).__init__()
        self.debug = debug
        if self.debug: print "args:",args
        arg_list = command_and_args.split()
        self.command=arg_list
        if self.debug: print "command=",self.command
        #self.daemon = True
        
    def __call__(self):
        if self.debug: print "Call to __call__"
        self.run()
        
    def run(self):
        if self.debug: 
            print "In {}".format(self.name)
            print "Calling",self.command

        proc = subprocess.Popen(self.command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        self.__handleOutput__(proc)
        if self.debug:
            print "Exiting",multiprocessing.current_process().name
            print 'In %s' % self.name
        return

    def __handleOutput__(self,proc):
        for line in iter(proc.stdout.readline, b''):
            print line,

In [131]:
# Execute command on remote (or local) host in background, using exec_remote.sh and BashExecutor instance.
# Command can be any command or executable file name.
# Arguments must be in a string separated with spaces.
def RemoteExec(hosts, command, args="", debug=False):
    package_directory = os.path.dirname(os.getcwd())
    scripts_location=os.path.realpath(os.path.join(package_directory,"mlframe","scripts"))
    exec_remote_script="exec_remote.sh"
    script_path = os.path.join(scripts_location,exec_remote_script)
    
    command_script_path = os.path.join(scripts_location,command)
    if debug: print "Test if script file exists in RemoteExec",command_script_path
    if os.path.isfile(command_script_path):
        command = command_script_path
    
          
    jobs = []
    if type(hosts) is str:
        hosts = [hosts]
    for host in hosts:
        print host
        if host == "" or host == "localhost" or host == "127.0.07":
            args = " ".join([command,args])
        else:
            args = " ".join([script_path, host, command, args])
        if debug: print "args",args
        d = BashExecutor(args, debug=debug)
        jobs.append(d)        
        d.start()
        
    #for d in jobs:
    #    d.join()
    
    print "finished"

In [132]:
# Нужно выделять имя скрипта в отдельный аргумент, чтобы можно было отличить его от других команд
RemoteExec("mouse","test.sh"," -a -b")
RemoteExec("localhost","date")

mouse
finished
localhost
finished
Wed Mar  7 12:21:04 JST 2018
REMOTE_ADDRESS=mouse
SCRIPT_FILE_PATH=/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
PWD: /Users/peterbryzgalov/work/ML/mlframework/mlframe
REMOTE_COMMAND=/Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh
exec_remote.sh arguments: -a -b
Run /Users/peterbryzgalov/work/ML/mlframework/mlframe/scripts/test.sh -a -b on mouse
Running test command on mouse
mouse-a:1
mouse-a:2
mouse-a:3
mouse-a:4
mouse-a:5
mouse-a:6
Exit code 0
